# Code for Publication "Enhancing ICD-Code-Based Case Definition for Heart Failure Using Electronic Medical Record Data"

# Import modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn import tree
import graphviz 
import xgboost as xgb

import re
import datetime
import pickle
import xml.etree.ElementTree as ET
from os import listdir
from itertools import islice

# Process documents with cTAKES and parse output 

## Process Documents with cTAKES

In [ ]:
#Specifications to pass to cTAKES

#Your UMLS credentials
username = "***Your UMLS Username***"
password = "***Your UMLS Password***"

#Path to cTAKES clinical pipeline ***REPLACE WITH YOUR PATH***
pipeline = "~/Resources/apache-ctakes-4.0.0/bin/runClinicalPipeline.sh"

#Folder containing the documents you want processed, to simplify linkage I used the chart number (RHRN) 
#as the document name: RHRN.txt
inFolder = " -i data/DischargeSummaries/Text"

#Folder to put the annotated documents in
outFolder = " --xmiOut data/DischargeSummaries/cTAKESoutput/"

#UMLS credentials
UMLScred = f" --user {username} --pass {password}"

#Final Shell command to execute
cmd = pipeline + inFolder + outFolder + UMLScred

!$cmd

## Parse cTAKES output 

- create a dataframe where each row is a discharge summary
- each column a CUI
- each element is the number of times a given CUI appeared in the discharge summary, non-negated or uncertain and referring to the patient

In [ ]:
floc = "data/DischargeSummaries/cTAKESoutput/"
cDocs = listdir(floc)
boc = pd.DataFrame()

for count, doc in enumerate(cDocs):
    tree = ET.parse(floc + doc) 
    root = tree.getroot()
        
    RHRN = doc.split('.')[0]
    
#     boc.loc[count] = 0
#     boc['RHRN'].loc[count] =RHRN
    
    concepts = root.findall(".//*[@ontologyConceptArr]")
    concepts = [x.attrib for x in concepts]
    concepts = pd.DataFrame(concepts)
    concepts = concepts[(concepts['subject'] == 'patient') & (concepts['polarity'] == '1')]
    
    cuis = root.findall(".//*[@cui]")
    tmp = {'RHRN':RHRN}
    for cui in cuis:
        concept_id = cui.attrib['{http://www.omg.org/XMI}id']
        if (concepts['ontologyConceptArr'].str.contains(concept_id).sum()) > 0:
            ind = cui.attrib['cui']
            try:
                tmp[ind]+=1
            except:
                tmp[ind] = 1
            
        
    boc = boc.append(tmp,ignore_index=True)
    
    if (count % 50)==0:
        print(count)

# Merge cTAKES labels with DAD and Chart Review data

## Merge output with DAD

In [ ]:
dad = pd.read_csv('data/DAD.csv',low_memory=False)
dad['RHRN'] = dad['RHRN'].astype(str)
boc = boc.merge(dad,on='RHRN')

## Merge with Chart review

In [ ]:
cr = pd.read_excel('data/ChartRev.xlsx',sheet_name='FULLDATA')
boc = boc.merge(cr,on='RHRN')

### Remove uncertain cases

In [ ]:
boc =boc.loc[boc['CHF present'] != 'Maybe']

### Modify labels so 1 = CHF present and 0 = CHF not present

In [ ]:
boc['CHF present'] = (boc['CHF present'] == 'Yes').astype(int)

# Create columns to stratify on 

## Create column with patient ages

In [ ]:
BoC.columns[BoC.columns.str.contains('date',flags=re.IGNORECASE)]

In [ ]:
BoC['BIRTHDATE']= pd.to_datetime(BoC['BIRTHDATE'],format='%Y%m%d')
BoC['ADMITDATE']= pd.to_datetime(BoC['ADMITDATE'],format='%Y%m%d')

# BoC['birthdate']= pd.to_datetime(BoC['birthdate'],format='%Y%m%d')
# BoC['admitdate']= pd.to_datetime(BoC['admitdate'],format='%Y%m%d')
# (df.fr-df.to).astype('timedelta64[h]')
BoC['Age'] = (BoC['ADMITDATE'] - BoC['BIRTHDATE']).astype('timedelta64[Y]')


## Create a column indicating whether the patient died in hospital

In [ ]:
BoC['died'] = (BoC['Chart_Disp'] == 'Died')

# BoC['died'] = (BoC['Discharge disposition?'] == 'Died')
BoC['died'].sum()

## Import Provider Type Dictionary to Stratify Results by Whether they're surgical patients or not

In [ ]:
ProviderType = pd.read_csv('/Volumes/Projects/Elliot/ProviderTypeDic.csv')
ProviderType

In [ ]:
ll = ProviderType['PRVDR_SVC'].str.contains('Surgery', flags=re.IGNORECASE)

surg = ProviderType['DOCSVC'].loc[ll]
surg="|".join([str(x) for x in surg])
surg

# Analyze Data

## Define some helper functions

In [ ]:
#always round .5 up
import decimal
context = decimal.getcontext()
context.rounding = decimal.ROUND_HALF_UP

In [ ]:
def format_out(func):
    def standard_round(*args):
        out = [int(round(decimal.Decimal(x*100), 0)) for x in func(*args)]
        return f"{out[0]}({out[0]-out[1]}-{out[0]+out[1]})"
    return standard_round

@format_out
def precision(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    p = sum(np.where(predicted & actual,1,0))/sum(predicted)
    return p, ci(p,sum(predicted))

@format_out
def recall(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    r = sum(np.where(predicted & actual,1,0))/sum(actual)
    return r, ci(r,sum(actual))

@format_out
def accuracy(predicted,actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    a = sum(np.where(predicted==actual,1,0))/len(actual)
    return a, ci(a,len(actual))

@format_out
def specificity(predicted,actual):
    predicted = np.array(predicted)
    predicted = 1 - predicted
    actual = np.array(actual)
    actual = 1 - actual
    s = sum(np.where(predicted & actual,1,0))/sum(actual)
    return s, ci(s,sum(actual))

@format_out
def NPV(predicted,actual):
    predicted = np.array(predicted)
    predicted = 1 - predicted
    actual = np.array(actual)
    actual = 1 - actual
    n = sum(np.where(predicted & actual,1,0))/sum(predicted)
    return n, ci(n,sum(predicted))

# 95% confidence intervals
def ci(p,N):
    return 1.96*(p*(1-p)/N)**0.5


def stats(predicted,actual):
    print("Sample Size = ", len(actual))
    print("Positive Cases = ",sum(actual))
    print("Cases Labeled Positive = ", sum(predicted))
#     print("Negative Cases = ", sum(1-actual))
#     print("Cases Labeled Negative = ", sum(1-predicted))    
    
    print("Recall = ",recall(predicted,actual))
    print("Specificity = ",specificity(predicted,actual))    
    print("Precision = ",precision(predicted,actual))
    print("NPV = ",NPV(predicted,actual))    
    print("Accuracy = ",accuracy(predicted,actual))    

## Create an ICD-10 case defintion column

In [ ]:
# Select out all columns with ICD diagnoses codes
cols = boc.columns
codes = cols[cols.str.contains('dxcode',flags=re.IGNORECASE)]

# Create a column containing all diagnoes codes concatenated together
boc['AllCodes']=boc[codes].fillna("").apply(lambda row: " ".join([str(x) for x in row]),axis=1)

# Create a column that equal 1 if it satisfies the ICD-10 CHF definition and 0 otherwise
boc['ICD CHF'] = (boc['AllCodes'].str.contains('I099|I110|I130|I132|I255|I420|I425|I426|I427|I428|I429|I43|I50|P290')).astype(int)

## Feature selection with XGBoost

### Select all CUIs as predictors

In [ ]:
cuis = boc.columns.str.contains('^C\d')

X = boc.loc[:,cuis]
Y = boc['CHF present']

### Specify parameters

In [ ]:
xgb_model = xgb.XGBClassifier()

parameters = {'nthread':[32], #when use hyperthread, xgboost may become slower
              'lambda':[0,0.5,1,2], #L2 regularization term on weights
              'alpha':[0,0.5,1,2], #L1 regularization term on weights 
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [3,5,6],
              'min_child_weight': [4,8,16],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [10,100,500,1000], #number of trees, change it to 1000 for better results
              'seed': [42]}

scoring = 'roc_auc'
kf = KFold(n_splits=5,shuffle=True,random_state=42)

### Create 5 Cross-Validated XGBoost models

In [ ]:
for fold, indices in enumerate(kf.split(X)):
    
    clf = GridSearchCV(xgb_model, parameters, n_jobs=6,
                       cv=10, 
                       scoring=scoring,
                       verbose=2, refit=True)

    X_train = X.iloc[indices[0]]
    Y_train = Y.iloc[indices[0]]
    clf.fit(X_train, Y_train)
    filename = f"models/CHF-Model-FOLD-{fold}-CaseIdent-XGBoost-AllCUIs-SubjNegUncer.pkl"
    pickle.dump(clf, open(filename, 'wb'))

### Get stats for all folds

In [ ]:
folds = 5
kf = KFold(n_splits=folds,shuffle=True,random_state=42)
kf.get_n_splits(X)
rec = []
prec = []
for i in range(folds):
    train_ind, test_ind = next(islice(kf.split(X),i,i+1))
    X_samp = X.iloc[test_ind]
    Y_samp = Y.iloc[test_ind]
    
    filename = f'models/CHF-Model-FOLD-{i}-CaseIdent-XGBoost-AllCUIs-NoICD11-SubjNegUncer.pkl'
    clf = pickle.load(open(filename, 'rb'))
    print(f'Fold {i} ---------------')
    pred = clf.predict(X_samp)
    rec.append(float(recall(pred,Y_samp)[0:2]))
    prec.append(float(precision(pred,Y_samp)[0:2]))
    stats(pred,Y_samp)

print('mean recall', sum(rec)/folds)
print('mean precision', sum(prec)/folds)

### Compare feature importances

In [ ]:
%matplotlib inline

In [ ]:
folds = 5

nf = 10

tfs = {}

for i in range(folds):
    filename = f'models/CHF-Model-FOLD-{i}-CaseIdent-XGBoost-AllCUIs-SubjNegUncer.pkl'
    clf = pickle.load(open(filename, 'rb'))    
    
    tg = clf.best_estimator_.get_booster().get_score(importance_type= "gain")
    tg = pd.Series(tg)
    tg.sort_values(ascending=False,inplace=True)
    top_features = tg.iloc[:nf]
    # top_features

    tfs[i] = set(tg.index[:nf])

    pos = np.arange(top_features.shape[0])
    plt.subplot(folds, 1, i+1)
    plt.bar(pos,top_features)
#     plt.xticks(pos,top_features.index,rotation=-45)
    plt.ylabel("Feature Importance")
    plt.xlabel("Feature Rank")

### Select features that all appear in the top nf features

In [ ]:
agree = {}
for tf in tfs:
    try:
        agree = agree.intersection(tfs[tf])
    except:
        agree = tfs[tf]
        
agree

## Fit Decision tree using selected features

### Select features to use

In [ ]:
# After creating 5 different optimized XGBoost models using 5-fold cross-val (each optimized using 10-fold cross-val) and taking the features that are in the top 10 most important in all of the models

X = BoC[['C0016860', 'C0018801', 'C0018802', 'C0054836', 'C0277785', 'C0699992']]

In [ ]:
# Taking only the features from above that result in a change in classification
X = BoC[['C0018801', 'C0018802']]

In [ ]:
X.fillna(value=0,inplace=True)
Y = boc['CHF present']

### Create a Test/Train Split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y,random_state=42)

### Specify parameters

In [ ]:
parameters = {'criterion':['gini','entropy'],'max_depth':range(3,6),'min_samples_leaf':[3,5,10,20]}
scoring = 'roc_auc'

### Fit Tree Using Cross-Validation

In [ ]:
scoring = 'roc_auc'

clf = GridSearchCV(tree.DecisionTreeClassifier(splitter = 'best',random_state=42), 
                   parameters, cv = 10, 
                   n_jobs=4, scoring=scoring)

clf.fit(X_train, Y_train)

### Visualize graph

In [ ]:
import graphviz 

In [ ]:
mdl = clf.best_estimator_
dot_data = tree.export_graphviz(mdl, feature_names=X.columns,out_file=None,filled=True, rounded = True)
graph = graphviz.Source(dot_data)
graph

# Get stats for all models

## Select Data

In [ ]:
#All Data
# X_samp = X
# Y_samp = Y

#Test set
X_samp = X_test
Y_samp = Y_test

### ICD-10 Algorithm

In [ ]:
stats(boc['ICD CHF'],boc['CHF present'])

### Decision Tree

In [ ]:
pred = clf.predict(X_samp)
stats(pred,Y_samp)

### Combined

In [ ]:
pred = clf.predict(X_samp)
ICD = boc['ICD CHF']
combine = (pred | ICD.loc[X_samp.index]) 

In [ ]:
stats(combine,Y_samp)

## Stratified by Service (Surgical vs Not)

### Surgical

In [ ]:
# Select surgical patients
ll = boc['DOCSVC1'].astype(str).str.contains(surg)

#### ICD-10 Algorithm

In [ ]:
stats(boc['ICD CHF'].loc[ll],boc['CHF present'].loc[ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[ll])
stats(pred,Y_samp.loc[ll])

#### Combine

In [ ]:
stats(combine.loc[ll],Y_samp.loc[ll])

### Non-Surgical

#### ICD-10 Algorithm

In [ ]:
stats(boc['ICD CHF'].loc[~ll],boc['CHF present'].loc[~ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[~ll])
stats(pred,Y_samp.loc[~ll])

#### Combine

In [ ]:
stats(combine.loc[~ll],Y_samp.loc[~ll])

## Stratified by Age (65 and older vs Under 65)

In [ ]:
# Select patients 65 and older
ll = boc['Age'] > 64

### Older

#### ICD-10 Algorithm

In [ ]:
stats(boc['ICD CHF'].loc[ll],boc['CHF present'].loc[ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[ll])
stats(pred,Y_samp.loc[ll])

#### Combine

In [ ]:
stats(combine.loc[ll],Y_samp.loc[ll])

### Younger

#### ICD-10 Algorithm

In [ ]:
stats(boc['ICD CHF'].loc[~ll],boc['CHF present'].loc[~ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[~ll])
stats(pred,Y_samp.loc[~ll])

#### Combine

In [ ]:
stats(combine.loc[~ll],Y_samp.loc[~ll])

## Stratified by Mortality (Died vs Survived)

In [ ]:
# Select patients who died
ll = boc['died']

### Died

#### ICD-10 Algorithm

In [ ]:
stats(boc['ICD CHF'].loc[ll],boc['CHF present'].loc[ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[ll])
stats(pred,Y_samp.loc[ll])

#### Combined

In [ ]:
stats(combine.loc[ll],Y_samp.loc[ll])

### Survived

#### ICD-10 Algorithm

In [ ]:
stats(boc['ICD CHF'].loc[~ll],boc['CHF present'].loc[~ll])

#### Decision Tree

In [ ]:
pred =clf.predict(X_samp.loc[~ll])
stats(pred,Y_samp.loc[~ll])

#### Combined

In [ ]:
stats(combine.loc[~ll],Y_samp.loc[~ll])